## Azure ML GAN Expriment

To use this notebook, you need to download `config.json` file from Azure ML Workspace and place it in this folder. This will allow us to get the workspace reference right away:

In [1]:
dataset = 'd:/data/wikiart/wikiart/impressionism'

In [2]:
from azureml.core import Workspace

try:
    ws = Workspace.from_config()
    print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')
    print('Library configuration succeeded')
except:
    print('Workspace not found')

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.0.83 (c:\winapp\miniconda3\lib\site-packages), Requirement.parse('azureml-core==1.0.79.*'), {'azureml-telemetry'}).
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


AzMLWorkspace	northeurope	AzureMLGroup	northeurope
Library configuration succeeded


Then make sure we have the compute cluster. If the cluster does not exist - we will create it programmatically!

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cluster_name = "AzMLGPUCompute"

# Verify that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6',
                                                           vm_priority='lowpriority',
                                                           min_nodes=0,
                                                           max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


Now upload the images dataset into the Azure ML Workspace:

In [4]:
ds = ws.get_default_datastore()
ds.upload(dataset, target_path='dataset', overwrite=True)

Uploading an estimated of 617 files
Uploading d:/data/faces\face0.jpg
Uploading d:/data/faces\face1.jpg
Uploading d:/data/faces\face10.jpg
Uploading d:/data/faces\face100.jpg
Uploading d:/data/faces\face101.jpg
Uploading d:/data/faces\face102.jpg
Uploading d:/data/faces\face103.jpg
Uploading d:/data/faces\face104.jpg
Uploading d:/data/faces\face105.jpg
Uploading d:/data/faces\face106.jpg
Uploading d:/data/faces\face107.jpg
Uploading d:/data/faces\face108.jpg
Uploading d:/data/faces\face109.jpg
Uploading d:/data/faces\face11.jpg
Uploading d:/data/faces\face110.jpg
Uploading d:/data/faces\face111.jpg
Uploading d:/data/faces\face114.jpg
Uploading d:/data/faces\face115.jpg
Uploading d:/data/faces\face116.jpg
Uploading d:/data/faces\face117.jpg
Uploading d:/data/faces\face118.jpg
Uploading d:/data/faces\face119.jpg
Uploading d:/data/faces\face120.jpg
Uploading d:/data/faces\face121.jpg
Uploading d:/data/faces\face124.jpg
Uploading d:/data/faces\face126.jpg
Uploading d:/data/faces\face127.jp

Uploaded d:/data/faces\face157.jpg, 63 files out of an estimated total of 617
Uploading d:/data/faces\face204.jpg
Uploaded d:/data/faces\face189.jpg, 64 files out of an estimated total of 617
Uploading d:/data/faces\face205.jpg
Uploaded d:/data/faces\face158.jpg, 65 files out of an estimated total of 617
Uploading d:/data/faces\face206.jpg
Uploaded d:/data/faces\face159.jpg, 66 files out of an estimated total of 617
Uploading d:/data/faces\face207.jpg
Uploaded d:/data/faces\face161.jpg, 67 files out of an estimated total of 617
Uploading d:/data/faces\face208.jpg
Uploaded d:/data/faces\face19.jpg, 68 files out of an estimated total of 617
Uploading d:/data/faces\face210.jpg
Uploaded d:/data/faces\face160.jpg, 69 files out of an estimated total of 617
Uploading d:/data/faces\face211.jpg
Uploaded d:/data/faces\face163.jpg, 70 files out of an estimated total of 617
Uploading d:/data/faces\face212.jpg
Uploaded d:/data/faces\face162.jpg, 71 files out of an estimated total of 617
Uploading d

Uploaded d:/data/faces\face255.jpg, 140 files out of an estimated total of 617
Uploading d:/data/faces\face29.jpg
Uploaded d:/data/faces\face256.jpg, 141 files out of an estimated total of 617
Uploading d:/data/faces\face290.jpg
Uploaded d:/data/faces\face257.jpg, 142 files out of an estimated total of 617
Uploading d:/data/faces\face291.jpg
Uploaded d:/data/faces\face254.jpg, 143 files out of an estimated total of 617
Uploading d:/data/faces\face292.jpg
Uploaded d:/data/faces\face259.jpg, 144 files out of an estimated total of 617
Uploading d:/data/faces\face293.jpg
Uploaded d:/data/faces\face26.jpg, 145 files out of an estimated total of 617
Uploading d:/data/faces\face294.jpg
Uploaded d:/data/faces\face260.jpg, 146 files out of an estimated total of 617
Uploading d:/data/faces\face298.jpg
Uploaded d:/data/faces\face262.jpg, 147 files out of an estimated total of 617
Uploading d:/data/faces\face299.jpg
Uploaded d:/data/faces\face261.jpg, 148 files out of an estimated total of 617
Upl

Uploaded d:/data/faces\face352.jpg, 219 files out of an estimated total of 617
Uploading d:/data/faces\face389.jpg
Uploaded d:/data/faces\face353.jpg, 220 files out of an estimated total of 617
Uploading d:/data/faces\face39.jpg
Uploaded d:/data/faces\face355.jpg, 221 files out of an estimated total of 617
Uploaded d:/data/faces\face354.jpg, 222 files out of an estimated total of 617
Uploading d:/data/faces\face390.jpg
Uploading d:/data/faces\face391.jpg
Uploaded d:/data/faces\face357.jpg, 223 files out of an estimated total of 617
Uploading d:/data/faces\face392.jpg
Uploaded d:/data/faces\face356.jpg, 224 files out of an estimated total of 617
Uploading d:/data/faces\face393.jpg
Uploaded d:/data/faces\face36.jpg, 225 files out of an estimated total of 617
Uploading d:/data/faces\face394.jpg
Uploaded d:/data/faces\face362.jpg, 226 files out of an estimated total of 617
Uploaded d:/data/faces\face363.jpg, 227 files out of an estimated total of 617
Uploading d:/data/faces\face395.jpg
Upl

Uploaded d:/data/faces\face434.jpg, 298 files out of an estimated total of 617
Uploading d:/data/faces\face464.jpg
Uploaded d:/data/faces\face435.jpg, 299 files out of an estimated total of 617
Uploading d:/data/faces\face465.jpg
Uploaded d:/data/faces\face436.jpg, 300 files out of an estimated total of 617
Uploading d:/data/faces\face466.jpg
Uploaded d:/data/faces\face438.jpg, 301 files out of an estimated total of 617
Uploading d:/data/faces\face467.jpg
Uploaded d:/data/faces\face437.jpg, 302 files out of an estimated total of 617
Uploading d:/data/faces\face468.jpg
Uploaded d:/data/faces\face439.jpg, 303 files out of an estimated total of 617
Uploading d:/data/faces\face469.jpg
Uploaded d:/data/faces\face44.jpg, 304 files out of an estimated total of 617
Uploading d:/data/faces\face47.jpg
Uploaded d:/data/faces\face440.jpg, 305 files out of an estimated total of 617
Uploading d:/data/faces\face470.jpg
Uploaded d:/data/faces\face441.jpg, 306 files out of an estimated total of 617
Upl

Uploaded d:/data/faces\face510.jpg, 371 files out of an estimated total of 617
Uploading d:/data/faces\face548.jpg
Uploaded d:/data/faces\face511.jpg, 372 files out of an estimated total of 617
Uploading d:/data/faces\face549.jpg
Uploaded d:/data/faces\face513.jpg, 373 files out of an estimated total of 617
Uploading d:/data/faces\face55.jpg
Uploaded d:/data/faces\face512.jpg, 374 files out of an estimated total of 617
Uploading d:/data/faces\face550.jpg
Uploaded d:/data/faces\face514.jpg, 375 files out of an estimated total of 617
Uploading d:/data/faces\face551.jpg
Uploaded d:/data/faces\face515.jpg, 376 files out of an estimated total of 617
Uploading d:/data/faces\face552.jpg
Uploaded d:/data/faces\face516.jpg, 377 files out of an estimated total of 617
Uploading d:/data/faces\face553.jpg
Uploaded d:/data/faces\face52.jpg, 378 files out of an estimated total of 617
Uploading d:/data/faces\face554.jpg
Uploaded d:/data/faces\face524.jpg, 379 files out of an estimated total of 617
Upl

Uploaded d:/data/faces\face592.jpg, 444 files out of an estimated total of 617
Uploading d:/data/faces\face621.jpg
Uploaded d:/data/faces\face593.jpg, 445 files out of an estimated total of 617
Uploading d:/data/faces\face626.jpg
Uploaded d:/data/faces\face594.jpg, 446 files out of an estimated total of 617
Uploading d:/data/faces\face629.jpg
Uploaded d:/data/faces\face595.jpg, 447 files out of an estimated total of 617
Uploading d:/data/faces\face63.jpg
Uploaded d:/data/faces\face596.jpg, 448 files out of an estimated total of 617
Uploading d:/data/faces\face630.jpg
Uploaded d:/data/faces\face597.jpg, 449 files out of an estimated total of 617
Uploading d:/data/faces\face631.jpg
Uploaded d:/data/faces\face598.jpg, 450 files out of an estimated total of 617
Uploading d:/data/faces\face632.jpg
Uploaded d:/data/faces\face60.jpg, 451 files out of an estimated total of 617
Uploaded d:/data/faces\face599.jpg, 452 files out of an estimated total of 617
Uploaded d:/data/faces\face6.jpg, 453 f

Uploaded d:/data/faces\face676.jpg, 522 files out of an estimated total of 617
Uploading d:/data/faces\face706.jpg
Uploaded d:/data/faces\face677.jpg, 523 files out of an estimated total of 617
Uploading d:/data/faces\face707.jpg
Uploaded d:/data/faces\face678.jpg, 524 files out of an estimated total of 617
Uploading d:/data/faces\face708.jpg
Uploaded d:/data/faces\face679.jpg, 525 files out of an estimated total of 617
Uploading d:/data/faces\face709.jpg
Uploaded d:/data/faces\face68.jpg, 526 files out of an estimated total of 617
Uploading d:/data/faces\face71.jpg
Uploaded d:/data/faces\face680.jpg, 527 files out of an estimated total of 617
Uploading d:/data/faces\face710.jpg
Uploaded d:/data/faces\face681.jpg, 528 files out of an estimated total of 617
Uploading d:/data/faces\face711.jpg
Uploaded d:/data/faces\face682.jpg, 529 files out of an estimated total of 617
Uploading d:/data/faces\face712.jpg
Uploaded d:/data/faces\face683.jpg, 530 files out of an estimated total of 617
Upl

Uploaded d:/data/faces\face87.jpg, 605 files out of an estimated total of 617
Uploaded d:/data/faces\face88.jpg, 606 files out of an estimated total of 617
Uploaded d:/data/faces\face89.jpg, 607 files out of an estimated total of 617
Uploaded d:/data/faces\face675.jpg, 608 files out of an estimated total of 617
Uploaded d:/data/faces\face90.jpg, 609 files out of an estimated total of 617
Uploaded d:/data/faces\face9.jpg, 610 files out of an estimated total of 617
Uploaded d:/data/faces\face92.jpg, 611 files out of an estimated total of 617
Uploaded d:/data/faces\face91.jpg, 612 files out of an estimated total of 617
Uploaded d:/data/faces\face93.jpg, 613 files out of an estimated total of 617
Uploaded d:/data/faces\face95.jpg, 614 files out of an estimated total of 617
Uploaded d:/data/faces\face98.jpg, 615 files out of an estimated total of 617
Uploaded d:/data/faces\face99.jpg, 616 files out of an estimated total of 617
Uploaded d:/data/faces\face691.jpg, 617 files out of an estimate

$AZUREML_DATAREFERENCE_0baa0ab97974434e99cf8cf9ab68871d

Now let us create training script:

In [75]:
%%writefile train_gan.py
# KeraGAN trainer script

import argparse
import keragan
import keras
import os
import glob
from azureml.core.run import Run
import matplotlib.pyplot as plt

print("KeraGAN Trainer, version {}".format(keragan.__version__))

run = Run.get_context()

parser = argparse.ArgumentParser(description="KeraGAN Trainer")

parser.add_argument("--path",help="Azure ML Datastore dir")
parser.add_argument("--dataset",help="Dataset name within datastore")
parser.add_argument("--size",help="Image size to use", default=512, type=int)
parser.add_argument("--aspect_variance",help="Allowed aspect variance", default=0.5, type=float)
parser.add_argument("--model_path",help="Path to use for saving models", default='models')
parser.add_argument("--samples_path",help="Path to use for saving samples", default='samples')
parser.add_argument("--save_npy_path",help="Filename to save cached dataset for faster loading")
parser.add_argument("--limit",help="Limit # of images to use",type=int,default=None)
parser.add_argument("--batch_size",help="Minbatch size to use",type=int,default=128)
parser.add_argument("--save_interval",help="Epochs between saving models",type=int,default=100)
parser.add_argument("--save_img_interval",help="Epochs between generating image samples",type=int,default=100)
parser.add_argument("--print_interval",help="Epochs between printing",type=int,default=10)
parser.add_argument("--sample_images",help="View image sample",action='store_const',default=False,const=True)
parser.add_argument("--no_samples",help="Number of sample images to generate during training",type=int,default=10)
parser.add_argument("--latent_dim",help="Dimension of latent space",type=int,default=256)
parser.add_argument("--ignore_smaller",help="Ignore images smaller than required size",action='store_const',default=False,const=True)
parser.add_argument("--crop",help="Crop images to desired aspect ratio",action='store_const',default=False,const=True)
parser.add_argument("--epochs",help="Number of epochs to train",type=int,default=100)
parser.add_argument("--lr",help="Learning rate",type=float,default=0.0001)

args = parser.parse_args()

args.height = args.size
args.width = args.size
args.optimizer = None
args.path = os.path.join(args.path,args.dataset)

gan = keragan.DCGAN(args)
imsrc = keragan.ImageDataset(args)
imsrc.load()
train = keragan.GANTrainer(image_dataset=imsrc,gan=gan,args=args)

def callbk(tr):
    if tr.gan.epoch % 20 == 0:
        res = tr.gan.sample_images(n=3)
        fig,ax = plt.subplots(1,len(res))
        for i,v in enumerate(res):
            ax[i].imshow(v[0])
        run.log_image("Sample",plot=plt)

train.train(callbk)


Overwriting gantrain.py


Now let's submit the experiment to run:

In [7]:
from azureml.core import Experiment
#from azureml.train.estimator import Estimator
from azureml.train.dnn import TensorFlow

experiment_name = 'KeraGAN'
exp = Experiment(workspace=ws, name=experiment_name)
script_params = {
    '--path': ws.get_default_datastore(),
    '--dataset' : 'faces',
    '--model_path' : './outputs/models',
    '--samples_path' : './outputs/samples',
    '--batch_size' : 32,
    '--size' : 512,
    '--limit': 3000,
    '--epochs' : 10000
}

est = TensorFlow(source_directory='.',
                script_params=script_params,
                compute_target=cluster,
                entry_script='train_gan.py',
                use_gpu = True,
                conda_packages=['keras','tensorflow','opencv','tqdm','matplotlib'],
                #pip_packages=['imutils','opencv-python','git+https://github.com/shwars/keragan']
                pip_packages=['git+https://github.com/shwars/keragan@v0.0.1']
)

framework_version is not specified, defaulting to version 1.13.


In [8]:
run = exp.submit(est)

In [7]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
KeraGAN,KeraGAN_1584017036_95b6872c,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


### Downloading resulting images

After the experiment has completed, you can download resulting images to your local machine. If the experiment was long-running and the notebook session is lost, you can re-create it knowing the run id (which you can get from the portal). Otherwise you can use the same `run` variable from above (skip the cell below)

In [15]:
from azureml.core import Run
run = Run(experiment=exp,run_id='KeraGAN_1584048041337') # <-- provide run id from Azure Portal here
run.get_file_names()[:10]

['Sample_1584048230.png',
 'Sample_1584048271.png',
 'Sample_1584048311.png',
 'Sample_1584048353.png',
 'Sample_1584048396.png',
 'Sample_1584048438.png',
 'Sample_1584048479.png',
 'Sample_1584048520.png',
 'Sample_1584048562.png']

Следующая команда скачивает все сгенерированные сэмплы (которые наш скрипт помещал в директорию `outputs/samples`) на локальный компьютер. При этом в текущей директории появляется директория `outputs/samples` со всеми файлами.

In [20]:
run.download_files(prefix='outputs/samples')

In [18]:
# hyperdrive_run.cancel()